<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
# import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
import regex as rex
import standardize_library as standlib
from standardize_library import StandardizeNames
from unidecode import unidecode
import copy

import name_matching_lib as nmlib
import time
# from sklearn.feature_extraction.text import TfidfVectorizer
# from scipy.sparse import csr_matrix
# import sparse_dot_topn.sparse_dot_topn as ct  #Cosine Similarity

# from function_library import *
# from function_library2 import *

In [4]:
pd.set_option("display.max_rows", 999)

In [26]:
# either the sender or the recipient
# df = pd.read_csv('new_clean_output.csv',index_col = 0)
df = pd.read_csv('output_0211.csv')

# df = df.drop_duplicates(keep='first',subset=['Sent'])
# df = df.reset_index(drop=True)

from_list = df['From'].values.tolist()
to_list = df['To'].values.tolist()
cc_list = df['CC'].values.tolist()
print(len(from_list))

71143


In [77]:
np.savetxt("cc_list.txt", cc_list, fmt='%s') 

In [28]:
name = "Elyea, Janice, Lisenby, Katherine"

In [29]:
for el in cc_list:
    if pd.isnull(el): continue
    if rex.match(r'.?Elyea, Janice, Lisenby, Katherine', el):
        print("==> ", el)

==>   Elyea, Janice, Lisenby, Katherine, Baldwin, Angela, Lowe, Terry, 'Paige Carter-Smith', Worrell, Don, Carter-Smith, Janice, Goad, Reese
==>   Elyea, Janice, Lisenby, Katherine
==>   Elyea, Janice, Lisenby, Katherine
==>   Elyea, Janice, Lisenby, Katherine
==>   Elyea, Janice, Lisenby, Katherine, Baldwin, Angela, Lowe, Terry, 'Paige Carter-Smith', Worrell, Don, Carter-Smith, Janice, Goad, Reese
==>   Elyea, Janice, Lisenby, Katherine


In [30]:
cc_list;

In [31]:
# New class that collects all useful routines
sz = 1000
# sz = None
# stand = StandardizeNames(from_list[:sz], to_list[:sz], cc_list[:sz])
stand = StandardizeNames(from_list[:], to_list[:], cc_list[:])
stand.process()

In [32]:
stand.clean_names_without_emails;
len(stand.clean_names_with_emails), len(stand.clean_names_without_emails)
# len(stand.name_to_emails), len(stand.email_to_names)

(2008, 9048)

In [33]:
my_list = stand.clean_names_with_emails + stand.clean_names_without_emails
matches_df = nmlib.name_matches(my_list, cs_thresh=0.7, ngram=2)

In [34]:
matches_df.shape

(31, 3)

In [36]:
matches_df;

In [37]:
left = matches_df['left_side'].values
right = matches_df['right_side'].values
simil = matches_df['similarity'].values
mails = set(stand.clean_names_with_emails)
nomails = set(stand.clean_names_without_emails)
for l,r,s in zip(left, right, simil):
    if l in nomails and r in mails: 
        print(f"no mail/mail: %20s,  %20s, %14.2f" % (l, r, s))
    elif r in nomails and l in mails:
        print(f"no mail/mail: %20s,  %20s, %14.2f" % (r, l, s))

no mail/mail:    northjonesallison,     allisonnorthjones,           0.85
no mail/mail:          tantallison,           allisontant,           0.85
no mail/mail:          adamgoodsen,           adamgoodson,           0.83
no mail/mail:    allenthompsoncell,         allenthompson,           0.83
no mail/mail:          allisonscor,          allisonscott,           0.82
no mail/mail:   allmailboxesbewell,          allmailboxes,           0.81
no mail/mail: allisonbehrmanthompson,        allisonbehrman,           0.81
no mail/mail:          idathompson,        amandathompson,           0.80
no mail/mail:        thompsonallen,         allenthompson,           0.80
no mail/mail: allisonbehrmancappel,        allisonbehrman,           0.79
no mail/mail:              ataylor,           alanataylor,           0.78
no mail/mail:             adamcory,             adamcorey,           0.78
no mail/mail:    alexmedinabarnack,            alexmedina,           0.77
no mail/mail:          jananderson, 

In [ ]:
standlib.clean_name_better('5Gary Yordon (gary@govinc net')

In [38]:
# Scan all the names 
stand.clean_to_unclean_names()
lst = list(stand.clean_to_unclean.items())
lst.sort(key=lambda x: x[0])

In [40]:
nms = stand.clean_to_unclean
nms['alanataylor']

{" 'Alana Taylor'",
 ' Alana Taylor',
 'Alana Taylor',
 'Alana Taylor <ataylor@think3d.net',
 'Alana Taylor <ataylor@think3d.net>, Stefanie Kiedrowski <skiedrow@gmail.com>, Paige CarterSmith <paigecartersmith@gmail.com',
 'Alana Taylor [mailto:ATaylor@think3d.net]'}

In [41]:
nms['garyyordon']
# standlib.clean_name_better(' 5Gary Yordon')

{" 'Gary Yordon'",
 " 'Gary Yordon', 'Rick Oppenheim'",
 " 'Gary Yordon', Chuck Mitchell",
 " 'Gary Yordon', Gmax1 (gmax1@comcast.net)",
 " 'Gary Yordon', Wes Townson",
 " 'gary yordon'",
 ' 5Gary Yordon',
 ' 5Gary Yordon (gary@govinc net)',
 ' 5Gary Yordon (gary@govinc.net)',
 ' 5Gary Yordon(gary@govinc.net)',
 ' <Gary Yordon',
 ' Gary Yordon',
 ' Gary Yordon (gary@govinc.net)',
 ' Gary Yordon (gary@govinc.net), Erik Rowen',
 ' Gary Yordon (gary@govinc.net), Gary Yordon(gbyordon@gmail.com)',
 ' Gary Yordon (gary@zprgroup.com)',
 ' Gary Yordon (gbyordon@gmail.com)',
 ' Gary Yordon (gbyordon@gmail.com) Monday, February 12, 2018 at 4:57:45 PM Eastern Standard Time',
 ' Gary Yordon <gary@govinc.net',
 ' Gary Yordon <gary@zprgroup.com',
 ' Gary Yordon(gary@govinc.net)',
 ' Gary Yordon(gbyordon@gmail.com)',
 ' GaryYordon',
 ' Yordon, Gary',
 ' gary yordon',
 ' yordon, gary',
 "'Gary Yordon'",
 "'Gary Yordon' <gary@zprgroup.com",
 "'gary yordon'",
 '5Gary Yordon (gary@govinc.net)',
 'Gary Yo

In [42]:
nms['amandatease']

{' Tease, Amanda',
 ' Tease, Amanda <Amanda.Tease@talgov.com>',
 '"Tease, Amanda" <Amanda.Tease@talgov.com',
 'Tease, Amanda'}

In [43]:
if False:
    for i,el in enumerate(lst):
        if len(el[1]) > 3: 
            #print(f"___{el[0]}______  <<<  {len(el[1])}  >>>")
            pass
        else: 
            print(f"___{el[0]}______   {el[1]}  ")
        if i > 1000: break

# update From header

In [59]:
len(from_list)
with_emails = defaultdict(set)
without_emails = defaultdict(set)
count = 0
count_from_with_emails = 0
count_from_without_emails = 1
new_from_list = []

fd = stand.field_dict1
for el in from_list:
    if not fd[el]: 
        count += 1
        new_from_list.append('')
        continue
    name, email = fd[el]
    if email == '':
        without_emails[el] = name
        new_from_list.append(name)
    else:
        with_emails[el] = email
        new_from_list.append(email)

In [60]:
len(without_emails), len(with_emails), len(from_list), count

(1741, 3108, 71143, 777)

In [64]:
fd;

In [66]:
# The From without an email might not actually be real emails, but messages within the emails. Not sure about that. 
# 
df['new_from'] = new_from_list
len(new_from_list), df.shape

(71143, (71143, 16))

# update To header

In [70]:
for i in range(10):
    print("==> ", to_list[i])

==>  Brooks Hayes (brooks@culpeppercc.com)
==>  Coons, Kristen
==>  kim@inkbridge.com
==>  Faris, Alison
==>   andrew.gillum@talgov.com, anita.ﬂowers@talgov.com, rick.fernandez@talgov.com, Nancy Miller, scof, gil.ziﬀer@talgov.com, cur_s.richardson@talgov.com, deslogeb@leoncountyFL.gov, dozierk@leoncountyFl.gov, proctorb@leoncountyFL.gov, dailey@leoncountyFL.gov, Mary Ann Lindley
==>   Elias Mathes TDIA, Davis, Danielle, Paige Carter-Smith, downtownmarket@earthlink.net
==>  Alfano, Michael
==>   longV@leoncountyﬂ.gov, Mary Ann Lindley, proctorb@leoncountyFL.gov, Monday, February 12, 2018 at 7:15:45 PM Eastern Standard Time
==>  Chad Kittrell
==>  Sue Dick; Dana Noles; Whitney Weeks; Ed Edward Murray Jr.; Kristin Dozier; Cecilia Homison; lonnie.ballard@talgov.com; balogs@tcc.fl.edu; kgb@coloneybell.com; rbouthillier@stearnsweaver.com; jt@inkbridge.com; will@realestateinsync.com; paige@governanceinc.com; dcoburn@fsu.edu; betsy.couch@kccitallahassee.com; chris.curry@talgov.com; daileyj@leo

In [ ]:
len(to_list)
with_emails = defaultdict(set)
without_emails = defaultdict(set)
count = 0
count_from_with_emails = 0
count_from_without_emails = 1
new_from_list = []

fd = stand.field_dict1
for el in from_list:
    if not fd[el]: 
        count += 1
        new_from_list.append('')
        continue
    name, email = fd[el]
    if email == '':
        without_emails[el] = name
        new_from_list.append(name)
    else:
        with_emails[el] = email
        new_from_list.append(email)

# update Cc header

# Completed section based on class Standardize
---

In [280]:
print(stand.name_to_emails["adam corey"])

{'', 'mailtozacorey@gunster.com', 'rgrindler@101tally.com', 'acorey@gunster.com', 'adam@edisontally.com', 'mailtozacoreng101tally.com', 'adambcorey@gmail.com', 'mailtozacorey@101tally.com', 'adam@unconventionalstrategies.com', 'rmills@fsuoldschool.com', 'byronburroughs@yahoo.com', 'adam@101tallahassee.com', 'hlayerd@think3d.net', 'acorey@101tally.com', 'acorey@101tally.co'}


In [281]:
# field_dict, unique_names, field_to_clean, unrecognized_names 
field_dict = standlib.create_field_dict(from_list, to_list, cc_list)
field_dict1, unique_names, removed, unrecognized_names = standlib.clean_field_dict_values(field_dict)

# study seperately
# removed: remove all names (field content from From:, To:, cc:) with length greater than 20 (number can change)
# unrecognized names: 

In [ ]:
standlib.print_dict(field_dict1, 1)

In [ ]:
removed[0:5]

In [ ]:
# cleaned names: 
double_caps = []
unq = list(unique_names.keys())
#unq.sort(key=lambda x: x.lower().strip())
# Sort with respect to last name
unq.sort(key=lambda x: rex.sub(".*\s(\w+)$", r'\1', x.lower()))
print(len(unq))
for k in unq:
    n = standlib.join_prefix_to_stem(k)
    # Two capital letters in succession
    if rex.match(r'.*([A-Z])([A-Z]).*\Z', k):
        # print("Double Cap: ", k)
        double_caps.append(n)
    if  n != k:
        print(f"{k}_____{n}")
    #if  rex.match(r'\A\s?\w{1,}\ \w+\s?\Z', k, flags=rex.UNICODE):
    if 1:
        # print(f"___{k}___")
        pass
print("nb double caps: ", len(double_caps))

In [ ]:
field_dict1['Shane Moniz']
def analyze(name):
    print(f"field_dict1[{name}]: {field_dict1[name]}")
analyze('Shane Moniz')

## Abbreviations create issues. 
Perhaps the best way to define a name is to combine all the letters of the first, middle and last nmaes together and remove spaces, and lower case. Names are of secondary importance. It is the Emails that are important. So the goal might be to impute all  non-existent emails from the corresponding name if available. If not available, create a fake email.   Ultimately, 
* name_to_emails: For each name with no email, try and figure out the email from email_to_names. 
* This does not work. 
### Collect To/CC/From entries with no emails
* try to determine the emails from field_dict values
* start from email_to_names and name_to_emails. Note that these are cleaned names

In [ ]:
email_to_names, name_to_emails = standlib.compute_email_name_dicts(field_dict1)
len(email_to_names), len(name_to_emails)

In [ ]:
for k in name_to_emails.keys():
    if type(k) == tuple:
        print('tuple', k)

In [ ]:
standlib.print_dict(email_to_names, 10)

In [ ]:
standlib.print_dict(name_to_emails, 5)

In [ ]:
len(email_to_names), len(name_to_emails)

In [10]:
# List of field_dict elements with no associated emails. In other words, the From:/To:/CC: fields have 
#  entries with no email (i.e., no @). These are names with no email.
names_without_emails = standlib.get_names_without_emails_from_list(list(field_dict.keys()))
# list of tuples (field name, cleaned name)
names_without_emails1 = standlib.get_names_without_emails_from_dict_tuples(field_dict1)
len(names_without_emails), len(names_without_emails1)

[autoreload of standardize_library failed: Traceback (most recent call last):
  File "/Users/erlebach/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/erlebach/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Users/erlebach/opt/anaconda3/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Users/erlebach/opt/anaconda3/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 779, in exec_module
  File "<frozen importlib._bootstrap_external>", line 916, in get_code
  File "<frozen importlib._bootstrap_external>", line 846, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_rem

NameError: name 'standlib' is not defined

In [ ]:
list(field_dict.keys())[0:5]

In [ ]:
list(field_dict1.keys())[0:5]

In [ ]:
names_without_emails1[0:5]

In [ ]:
# How many clean names without email
clean_names_without_emails = set()
for el in names_without_emails1:
    clean_names_without_emails.add(el[1])
print(len(clean_names_without_emails))  #11,358 (instead of 17,224 unclean ones)

In [ ]:
missing_emails = set()
for name in names_without_emails1: #(dirty_name, clean_name)
    missing_emails.add(name[1])
missing_emails = list(missing_emails)
missing_emails.sort(key=lambda x: x)
missing_emails;

In [ ]:
names_without_emails1[100:110]

In [ ]:
recovered_emails = {}
print("init: len(missing_emails): ", len(missing_emails))
for name in missing_emails:
    emails = name_to_emails[name]
    if emails != {''}:
        if '' in emails:
            emails.remove('')
        recovered_emails[name] = emails
        
for name in recovered_emails.keys():
    missing_emails.remove(name)
        
print("len(recovered_emails): ", len(recovered_emails))
print("len(missing_emails): ", len(missing_emails))
names_without_emails = missing_emails
np.savetxt("missing_emails.txt", missing_emails, fmt="%s")

# Identification of similar names
* In this way, a name $n_1$ form the missing_emails list, will be paired with a name $n_2$ from the name_to_emails list. 
* I need a measure of similary between two names. Perhaps the overlap? Or first and last name with the corresponding first and last names? 

* I now have two lists: 
    * missing_emails (names without emails) 
    * name_to_mails (names with mails)
    * Use NamesMatch with two name lists: missing_emails and name_to_mails.keys(). Make each name into a list 
    * of 1 name. See what happens
* What I really want is a set of names. 
    * Look at email_to_names: Remove names that are obviously incorrect (how?)
    * What is left is an id (a set of names linked to a single individual)
    * I might not have the full id list. 
        * email1 ==> (name1a, name1b)
        * email2 ==> (name1a, name2b)
        * Therefore, the person's id is (name1a, name1b, name2b)
    * QUESTION: How to create an expanded id list? An id list has a corresponding email set (email1, email2, etc)
    * To each name without an email, Identify one or more emails, i.e., an email set. 
    * NEXT QUESTION: How to assign an email to which name (perhaps uniform random)? 

In [ ]:
names_with_mails = list(name_to_emails.keys())  # Remove names with no emails (TODO)
names_without_emails = missing_emails

In [ ]:
keys = list(name_to_emails.keys())
keys[-1]  # should be a string, not a sequence!

In [ ]:
# Create lists of lists
names_with_emails = []
for name in name_to_emails.keys():
    names_with_emails.append([name])
    
names_without_emails = []
for name in missing_emails:
    names_without_emails.append([name])

In [ ]:
len(names_with_mails), names_with_mails;

# Name Matching

---

In [ ]:
#List of names without emails
len(missing_emails)

In [ ]:
# list of names with emails
len(names_with_emails)

In [ ]:
total_list = missing_emails + names_with_mails
matches_df = nmlib.name_matches(missing_emails)

In [ ]:
missing_emails;
emails = list(email_to_names.keys())
emails;

In [ ]:
emails_prefix = []
# Create a list of names with email address from '@' removed
for email in emails:
    emails_prefix.append(rex.sub(r"@.*$", "", email))
emails_prefix;
np.savetxt("emails_prefix.csv", emails_prefix, fmt="%s")

In [ ]:
len(emails_prefix), len(missing_emails)

In [ ]:
#standlib.print_dict(name_to_emails)
for i, (name, emails) in enumerate(name_to_emails.items()):
    lst = [name] + list(emails)
    for i, l in enumerate(lst):
        lst[i] = l.lower()
    print("lst: ", lst)
    lst = ['shane moniz', 'smoniz@connandassociates.com']
    lst = ['shane moniz', 'sh. moniz@']
    # matches = nmlib.name_matches(missing_emails, cs_thresh=0.2)
    matches = nmlib.name_matches(lst, cs_thresh=0.1)
    # matches = nmlib.name_matches_nonsparse(lst, cs_thresh=0.2)
    print("matches: ", matches)
    print(".........................................")
    if i > 2: break
    

In [ ]:
np.dot

In [ ]:
# matches_df = nmlib.name_matches(names_with_mails)
matches_df = nmlib.name_matches(names_with_mails, cs_thresh=0.8)
# matches_df = nmlib.name_matches(emails_prefix, 0.6)
matches_df.shape, matches_df.tail(50)
name_to_emails["Tailor M"]

In [ ]:
em = emails_prefix[:155]
len(em)

In [ ]:
matches_df = nmlib.name_matches(em, 0.6)
matches_df

In [ ]:
matches_df = nmlib.name_matches_2(em, em[0:5], 0.6)
matches_df

In [ ]:
names_with_emails[-1]  # is a sequence. Should be a string!

In [ ]:
len(names_with_emails),len(names_without_emails)

In [ ]:
x = names_matcher(names_with_emails[0:200], names_without_emails[0:200]);

In [ ]:
x[0:10,:]

# standlib.print_dict(email_to_names, 5)

---

In [ ]:
# An email may have multiple names attached, with mispellings, etc. one name must be chosen. 
# WHAT is null_chosen_names? <<<<<
# Define notion of identity: 
##  email_to_names: email => set of names associated with that email (rightly or wrongly). 
## Remove all the names that are obviously wrong (using overlap for example): 
### choose name with max overlap with email, which is name_max
###          compute overlap between name_max and all other names in the set. 
##           keep the names with overlap beyond a threshhold

null_chosen_names, email_to_chosen_name, email_to_names_with_periods = \
    standlib.compute_email_to_chosen_name(email_to_names)

In [ ]:
standlib.print_dict(email_to_chosen_name, 5)

In [ ]:
len(email_to_chosen_name), len(email_to_names_with_periods), len(null_chosen_names)

In [ ]:
null_chosen_names[0:5]

# invert the dictionary; chosen_name_to_email  
* Check that to each name there is a single email
* One name can have multiple associated emails
* Store emails in lower case to minimize duplicates such as
    Gord@gmail.com and gord@gmail.com

In [ ]:
chosen_name_to_email = defaultdict(set)
for k,v in email_to_chosen_name.items():
    chosen_name_to_email[v].add(k.lower())

In [ ]:
standlib.overlap_str_str("patricia.sanzone@dep.state", "patricia.sanzone@dep.state.fl.us")

In [ ]:
standlib.print_dict(chosen_name_to_email, 10, max_length=10)

In [ ]:
# Starting from clean_names_without_emails, scan the list one element at a time and find 
# the corresponding email. How many will be found?
# Every person has a unique name, which is chosen_named
name_email = {} #defaultdict(list)
for name in clean_names_without_emails:
    email = chosen_name_to_email[name]
    if len(email) > 1 and len(email) < 10:
        print(f"{name}_____{email}")
        name_email[name] = email
# Values are sets. Some sets have two values. 

In [ ]:
name_to_emails['Ryan Grindler']
field_dict1['Ryan Grindler']

In [ ]:
len(name_email)
print(len(name_email), len(clean_names_without_emails))
name_email

In [ ]:
# How many names in chosen_name_to_email are included in names_without_emails: 

---


In [ ]:
len(names_without_emails), len(clean_names_without_emails)

In [ ]:
for name in clean_names_without_emails:
    print(name)

In [ ]:
name_to_emails

In [ ]:
len(names_without_emails1), len(unique_names)

In [ ]:
for k, v in unique_names.items():
    print(f"{k} ===>   {v}")
    print("------------------------------------------")

In [ ]:
# Names with no recognizable structure  first middle last or last, middle frst
# Will undergo further processing
# `unrecognized_names` are as found in To, From, CC email headers
# first with explodeCamel if there is a single word in camel case
# second by contrast with existing nanmes. For example, gjcoaker is probably G.J. Coaker . One way to tell with multiple consonnants following each other
print(len(unrecognized_names), type(unrecognized_names))
list(unrecognized_names)[0:10]

In [ ]:
len(field_dict)

In [ ]:
len(field_dict), len(name_to_emails), len(email_to_names)

In [ ]:
# All values of field_dict are (name, email) where `name` is the full name. 
for v in field_dict.values():
    if (len(v) != 2): 
        print(v)

In [ ]:
list(name_to_emails.keys())[0:10]

* ('Andrea', 'Rosser') ----- cleaned name: An ea Rosser
* ('Sheila', 'Williams') ----- cleaned name: Sheila Willia 
* ('Davisson Dunlap', 'III') ----- cleaned name: Davisson III
* name: ('Dietrich A', 'Simmons') ----- cleaned name: Dietrich Simmons
*
*
*
*
*
*
*
* Remove .com, .gov, etc. 
     * first name ends in ., last name begins in (com, gov, us, edu)

# `field_dict` has been enhanced with cleaned up names. How many different names are there? 
*   (from,to,cc)_list_element => (name, email)

In [ ]:
 email_to_names, name_to_emails = standlib.compute_email_name_dicts(field_dict1)

In [ ]:
len(email_to_names), len(email_to_names1), len(name_to_emails), len(name_to_emails1)
# WHY SO MUCH FEWER WITH THIS APPROACH?

In [ ]:
# make all emails lower case. For this, need to create new sets
# Go given that field_dict[k] = v,
# define v1 = set(), fill it up, and then field_dict[k] = v1

name_to_emails_lower = standlib.str_to_set_lower(name_to_emails)
name_to_emails_lower;

To each email corresponds a single name. Each name has multiple emails. 
Let us look at emails_to_name

In [ ]:
list(field_dict.keys())[20]
for k in field_dict.keys():
    if rex.match('.*Paige', k): 
        #print(k)
        pass

---
# End of new Analysis

In [ ]:
sorted(['A Taylor', 'Barber'])

In [ ]:
for k in sorted(name_to_emails.keys(), key=lambda x: x):
    if len(k) < 10:
        print(f"{k}___________{name_to_emails[k]}")

In [ ]:
for i, (k,v) in enumerate(name_to_emails.items()):
    if len(k) < 10:
        print(f"{k}_________________{v}")

In [ ]:
standlib.compute_email_to_chosen_name(email_to_names)

In [ ]:
for k,v in name_to_emails1.items():
    print(k,v)

In [ ]:
for k,v in name_to_emails1.items():
    if len(v) > 1 and len(v) < 10:
        
        
        print(k,v)

In [ ]:
# name_to_emails['Rodney'] ==> {'Austin', 'Lewis', 'Moore', 'Register'}

In [ ]:
# How could this happen? 
# Venus Childs ==> {'', 'venus.childs@talgov.com', 'Venus.Childs@talgov.com', 'Gary@zprgroup.com'}
for k,v in field_dict.items():
    if rex.match(r'\A.*Gary@zprgroup.com.*\Z', k):
        print(f"{k}----->{v}")

In [ ]:
field_dict

In [ ]:
for k,v in name_to_emails.items():
    print(f"{k} ==> {v}")

In [ ]:
unique = invert_dictionary(field_dict)
print(len(unique))  # way too high
print(len(field_dict))

In [ ]:
for k,v in invert_dictionary(field_dict).items():
    print(f"{k}_________{v}")  # v are keys from From, To, Cc lists
    
# What must be done: 
#  - identify all elements with a missing email
#    a) find email that match corresponding names
#    b) for names that cannot be matched, invent an email
#  - Make use of name_to_emails, emails_to_name

In [ ]:
len(unique_names), len(field_list)

In [ ]:
for k,v in unique_names.items():
    print(f"{k}-------------{v}\n")

In [ ]:
to_study_further = []
camel_exploded = []
left = []
unrecognized = []
for i, (k,v) in enumerate(name_to_emails.items()):
    name = k[0] + ' ' + k[1]
    name1 = standlib.clean_name(name)
    if name1 == name or name1 == 'unrecognized':
        unrecognized.append((name, name1))
        continue
    #print(f"name: {k} ----- cleaned name: {name1}")
    if len(name1.split()) == 2:
        name2 = standlib.explode_camelcase(name1)
        if len(name1) != len(name2):
            camel_exploded.append((name, name2))
        print(f"name: {name}<<< ------ cleaned name: {name2}<<<")
        to_study_further.append((name,name1))
    #if i > 5000: break 

In [ ]:
len(to_study_further), len(name_to_emails)

In [ ]:
to_study_further

In [ ]:
len(name_to_emails)

In [ ]:
name = 'Dr.Louis St. Petery'
standlib.remove_honorifics(name)

In [ ]:
standlib.explode_camelcase('gorfd MarkLaevine')

In [ ]:
for i, name in enumerate(to_study_further):
    name1 = standlib.clean_name(name)
    #name1 = standlib.clean_name(name1)
    if name1 == name or name1 == 'unrecognized':
        continue
    #print(f"name: {k} ----- cleaned name: {name1}")
    if len(name1.split()) > 2:
        print(f"name: {name} ------ cleaned name: {name1}")
    #if i > 5000: break 

name = 'hunterandharp. com'
standlib.remove_domains(name)

In [ ]:
print(len(field_dict))
for i, (k,v) in enumerate(field_dict.items()):
    if i > 30: break
    if len(k) > 80: continue
    if len(v) == 0: continue
    print(k,v)


In [ ]:
list(email_to_names.keys())
# Remove lines that are not emails, but save them in discarded list
non_emails = []

In [ ]:
from_dict = standlib.ge_search_from(from_list[0:10000], email_to_names, name_to_emails, unrecognized_names)
from_dict[3].add(3)
for k,v in from_dict.items():
    if len(k) < 50:
        print(k, v)

In [ ]:
def print_sorted_dict_keys(dct):
    lst = list(dct.values())
    lst.sort(key=lambda x: x[1])
    for k in lst:
        print(k)

In [ ]:
print_sorted_dict_keys(from_dict)

In [ ]:
for k,v in from_dict.items():
    if re.search('@', k):
        print(v)

In [ ]:
dlist = list(from_dict.items())
#print(dlist)
dlist.sort(key=lambda x: x[0][1][0])
for l in dlist:
    if len(l[0]) > 5000: continue
    print(l)
    
print(dlist[-1])

In [ ]:
for name, emails in name_to_emails.items():
        print(f"[{name}] => {emails}")

In [ ]:
for name in name_to_emails.keys():
    name1 = standlib.clean_name(name)
    print("-------------------------------")
    print(f"name: {name}, --------   cleaned: {cleaned}")

In [ ]:
standlib

In [ ]:
3

# Create dictionaries email -> name and name -> email
* Consider each element of name_to_emails. If there are two elements and one is empty, then name_email[name] = 'non_empty_email'
* If there are 3 elements, and one is empty, choose the email with most overlap between name and email owner. 
* If there is a single element that is empty, create a fake email:   last_first@fake.com
* If there are 4 elements, do not change them. Must be checked manually.
* Ignore the empty name the maps to many emails. 
* Consider an element of email_to_names. Ingore the email that is empty. 
* IF there is one entry in the values set, then email_name[email] = 'name'
* If there are two entries and one is empty, then email_name[email] = 'non-empty name'
* If there are three entries and one is empty, 


In [ ]:
null_chosen_names, email_to_chosen, email_to_names_with_periods \
    = standlib.compute_email_to_chosen_name(email_to_names)

In [ ]:
for i, (k, v) in enumerate(email_to_chosen.items()):
    if i < 5: print(f"email: {k} --- names: {v}")

for i, (k, v) in enumerate(email_to_names_with_periods.items()):
    if i < 5: print(f"email: {k} --- names with periods: {v}")

In [ ]:
null_chosen_emails, names_to_remove, name_to_chosen \
    = standlib.compute_name_to_chosen_email(name_to_emails)
# name_to_chosen_sort = list(name_to_chosen).sort(lambda x: x[0])
name_to_chosen_sort = list(name_to_chosen.items())
name_to_chosen_sort.sort(key=lambda x: x[0][0])

In [ ]:
for i, (k, v) in enumerate(name_to_chosen.items()):
    if i < 5: print(f"name: {k} --- emails: {v}")

names_to_remove[0:10]
null_chosen_emails[0:5]   # emails are null, so must be determined

In [ ]:
for name in null_chosen_emails:
    try:
        if name_to_chosen[name[0]]:
            print(name)
    except:
        pass
# None of the names in null_chosen emails are found in name_to_chosen

In [ ]:
emails_not_found_in_email_to_chosen, non_name_match, name_match \
    = standlib.round_trip_name(name_to_chosen, email_to_chosen)
    
names_not_found_in_name_to_chosen, non_email_match, email_match \
    = standlib.round_trip_email(email_to_chosen, name_to_chosen)

In [ ]:
email_to_chosen['abascom@wradvisors.com']

In [ ]:
name_to_chosen;

In [ ]:
# Create a list indexed by email, and collect the names. 
email_to_names = defaultdict(list)
for n,e in name_to_chosen.items():
    email_to_names[e].append(n)
# for k,v in email_to_names.items():
#     print(f"Email: {k} --- names: {v}")
len(email_to_names)

In [ ]:
# Add the names associated with the emails to the list in email_to_names 
# Either add new entries, and expand existing list


In [ ]:
len(name_to_chosen_sort)

In [ ]:
non_email_match
#non_name_match

In [ ]:
def remove_honorifics(name):
    name = re_honorifics.sub(' ', name, re.I)
    print(name)
    return re.sub(' +', ' ', name)

In [ ]:
name = 'Dr. Gordon E. dr. Bacher III'
remove_honorifics(name)

In [ ]:
names = (('laurie.', 'hartsfield'), ('Laurie', 'Hartsfield'))


In [ ]:
# Identify last name matches
matches = []
non_matches = []
for n1, n2 in non_name_match:
    if n1[1] == n2[1]:
        matches.append(n1)
    else:
        non_matches.append((n1,n2))
len(matches), len(non_matches)

In [ ]:
re_honorifics = re.compile(r'\b([\.,_-]|p\.?h\.?d\.?|j\.?r\.?|mr\.|ms\.|mrs\.|dr\.|iii)')
n = "usdr. G-_ordon iii dr. p.hd iii"
re_honorifics.findall(n)
re_honorifics.sub(' ', n)

In [ ]:
non_matches

In [ ]:
# Search whether names in null_chosen_emails are found in email_to_chosen
# for email, name in email_to_chosen.items():
#     print(email, name)

def search_null_chosen_emails():
    for name in null_chosen_emails:
        # search inefficiently, email_to_chosen
        for key in email_to_chosen.keys():
            if name[0] == email_to_chosen[key][0]:
                print("match: name: ", name)
                
# search_null_chosen_emails()
            
# No matches found. Therefore, none of the names in null_chosen_emails have corresponding emails. 
# Therefore, fake emails must be created. 
null_chosen_emails

In [ ]:
print("1: ",  email_to_chosen["CraigD@leoncountyfl.gov"] )
print("2: ",  email_to_chosen["CraigD@leoncountyﬂ.gov"] )
print("3: ",  email_to_chosen["Deborah.Craig@talgov.com"] )
print("4: ",  email_to_names["CraigD@leoncountyﬂ.gov"] )
print("5: ",  email_to_names["CraigD@leoncountyfl.gov"] )
print("6: ",  name_to_emails[('Deborah', 'Craig')] )

# One missing email: Deborah.Crag@talgov.com

In [ ]:
name_to_emails

In [ ]:
# How can these happen in non_email_match
# 'mailto:LBridges@flcities.com'),
# 'calendarno.ﬁca.on@google.com>', 'paige.tallahasseedowntown@gmail.com'),
# 

non_email_match
# email_match

In [ ]:
len(email_to_names_with_periods), email_to_names_with_periods;
# Incorrect removals: 
# 'cur.s@morethanbuildings.com': ('Cur.s', 'Whigham'),
# 'anita.favors.thompson@talgov.com': ('anita.favors.', 'thompson'),
#  'Chris.an.Doolin@talgov.com': ('Chris.an', 'Doolin')
# 3 out of 300 or 1% wrong removal. The reason is that the names were not capitalized in one case, 
#   an abbreviation was not capitalized (Cur.s instead of Cur.S), Same for Chris.an; the "an" should probably be capitalized.

In [ ]:
non_name_match

In [ ]:
email_to_chosen

In [ ]:
len(email_to_chosen)

In [ ]:
for email, names in email_to_names.items():
    if len(names) > 1 and len(names) < 10:
        print(f"[{email}] => {names}")

---
# COMPLETE FIRST ANALYSIS
## Below is Joey stuff
---

In [ ]:
people_list = []
named_email_list = []
name_list = []
unrecognized_names = set()
email_set = set()
people_set = set()
name_set = set()
# name_dict[('joe', 'blow')] = list of emails
name_dict = defaultdict(list) 

search_from_section(from_list)
print(len(people_list))
# search_to_section(to_list)
# print(len(people_list))
# search_cc_section(cc_list)
# print(len(people_list))

In [ ]:
people_set = set(people_list)
named_email_set = set(named_email_list)
name_set = set(name_list)

In [ ]:
people_list[0:2], named_email_list[0:2], name_list[0:2]

In [ ]:
unrecognized_names

In [ ]:
named_email_list

In [ ]:
standlib.analyze_from_list(from_list)

In [ ]:
len(new_from_list)

In [ ]:
len(from_list)

In [ ]:
x

In [ ]:
new_to_list = analyze_list_of_lists(to_list)
new_cc_list = analyze_list_of_lists(cc_list)

In [ ]:
len(to_list)

In [ ]:
len(new_to_lists)

In [ ]:
#new_to_lists[:10]
# for el in new_to_lists:
for i, el in enumerate(new_to_lists):
    ##if i > 5: break
    print(el)   # How can element of new_to_lists be empty?
    ell = el[0]
    if ell[0] == ' ' or ell[1] == ' ':
        print(ell)

In [ ]:
len(cc_list)

In [ ]:
len(new_cc_lists)

In [ ]:
new_cc_lists[:10]

In [ ]:
df_new = df.copy()
df_new['From'] = new_from_list
df_new['To'] = new_to_lists
df_new['CC'] = new_cc_lists
df_new


In [ ]:
df_new.to_csv('output_0211_name_standardized.csv', index=0)

In [ ]:
f = pd.read_csv('output_0211_name_standardized.csv')

In [ ]:
f.To.head()

In [ ]:
f.CC.head()

In [ ]:
eval(f.CC.head()[0])

---
# Not used

In [ ]:
email_to_names = defaultdict(set)
name_to_emails = defaultdict(set)
unrecognized_names = set()
field_dict = defaultdict(tuple)

# from_dict, to_dict, cc_dict are currently the identity mapping
# Eventually, each key will map to an email

standlib.ge_search_from(from_list, email_to_names, name_to_emails, unrecognized_names, field_dict)
print(len(field_dict))
standlib.ge_search_list_of_lists(to_list, email_to_names, name_to_emails, unrecognized_names, field_dict)
print(len(field_dict))
standlib.ge_search_list_of_lists(cc_list, email_to_names, name_to_emails, unrecognized_names, field_dict)
print(len(field_dict))

# All entries have a single element in the set. So sets were not required. 

# Some experiments with my new class

In [242]:
import unidecode
name = 'Ajontè Bush'
unidecode.unidecode(name)

'Ajonte Bush'

In [278]:
#if there is more than one comma, probably a list of anmes
names  = [
    '3 AE 45 Corey  HI KL Mathews', # looks like two names. Missing comma? AE: two initials
    'Adams Jennifer G',  # real name: Jennifer G. Adams? 
    'Ahmed Farhran R',  # I should keep all initials
    'Ana Cruz, Aide to Ortiz', # Should return Ana Cruz, not "aide cruz"
    'Ahmad Khalil Favors Sr', # should simply remove Sr
    'Roxanne Manning, AICP', # shoudl simply remove AICP
    'Chris Chadwell, Aide to Wiggs', #should return 'Chris Chadwell
    'Ajontè Bush', # should remove the accent
    'Ajowe Halsey Asha A', # should return the same
    'AIA Doug Shuler', # remove AIA
    'AICP Karen Jumonville', # remove AICP
    'Crawford, Aisha R', # return 'Aisha R Crawford'
    'Erlebacher, Gordon',
    'James "Jim" A. Snyder',
]

for name in names:
    #n = standlib.clean_name_better(name)
    n = standlib.clean_name_better(name)
    print(f"__{name}______{n}")


__3 AE 45 Corey  HI KL Mathews______corey mathews
__Adams Jennifer G______adams jennifer g
__Ahmed Farhran R______ahmed farhran r
__Ana Cruz, Aide to Ortiz______ aide to ortiz ana cruz
__Ahmad Khalil Favors Sr______ahmad khalil favors 
__Roxanne Manning, AICP______ roxanne manning
__Chris Chadwell, Aide to Wiggs______ aide to wiggs chris chadwell
__Ajontè Bush______ajonte bush
__Ajowe Halsey Asha A______ajowe halsey asha a
__AIA Doug Shuler______ doug shuler
__AICP Karen Jumonville______ karen jumonville
__Crawford, Aisha R______ aisha r crawford
__Erlebacher, Gordon______ gordon erlebacher
__James "Jim" A. Snyder______james jim a snyder
